# To export/import an opensearch index, either use directly opensearchpy

In [ ]:
from opensearchpy import OpenSearch
from opensearchpy.helpers import bulk
import pandas as pd

# Connect to OpenSearch
client = OpenSearch(hosts=[{"host": "localhost", "port": 9200}], http_compress=True)

# Query all documents from the index
index_name = "ind_name"
query = {"query": {"match_all": {}}}

# Use the search() method with scroll for large datasets
results = client.search(index=index_name, body=query, size=10000)
hits = results["hits"]["hits"]

# Convert to DataFrame
docs = [hit["_source"] for hit in hits]
df = pd.DataFrame(docs)

# Export to CSV
df.to_csv("ind_name.csv", index=False)

client.indices.delete(index=index_name)

actions = [
    {"_index": index_name, "_source": doc.dropna().to_dict()}
    for _, doc in df.iterrows()
]

bulk(client, actions)

# Or use awswrangler

In [ ]:
# Export and import an index in OpenSearch using AWS Wrangler (python 3.12.3)
# uv add awswrangler[opensearch] ipykernel awscli
# "aws configure" for awswrangler to work
import awswrangler as wr

oc = wr.opensearch.connect(host="localhost", port=9200)
results = wr.opensearch.search(
    oc,
    index="ind_name",
    size=10000,
)
results.to_csv("ind_name.csv", index=False)
wr.opensearch.delete_index(
    oc,
    index="ind_name",
)
wr.opensearch.index_csv(
    oc,
    path="ind_name.csv",
    index="ind_name",
)

# Or opensearch-py-ml

In [ ]:
import opensearch_py_ml as oml
from opensearch_py_ml.etl import csv_to_opensearch
import pandas as pd
from opensearchpy import OpenSearch


client = OpenSearch(hosts=[{"host": "localhost", "port": 9200}])
odf = oml.DataFrame(client, "ind_name")
odf.to_csv("ind_name.csv", index=False)

In [ ]:
client.indices.delete(index=index_name)

In [ ]:
csv_to_opensearch("ind_name.csv", client, "ind_name")